<a href="https://colab.research.google.com/github/ilhamsyah183/Machine-Learning-Documentation/blob/main/Latihan%20Text%20Classification%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd

df = pd.read_csv("/content/bbc-text.csv")
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
df_fix = df[['text', 'category']]
df_fix

,text,category
0,tv future in the hands of viewers with home th...,tech
1,worldcom boss left books alone former worldc...,business
2,tigers wary of farrell gamble leicester say ...,sport
3,yeading face newcastle in fa cup premiership s...,sport
4,ocean s twelve raids box office ocean s twelve...,entertainment
...,...,...
2220,cars pull down us retail figures us retail sal...,business
2221,kilroy unveils immigration policy ex-chatshow ...,politics
2222,rem announce new glasgow concert us band rem h...,entertainment
2223,how political squabbles snowball it s become c...,politics


In [ ]:
kategori = pd.get_dummies(df_fix.category)
df_baru = pd.concat([df_fix, kategori], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
...,...,...,...,...,...,...
2220,cars pull down us retail figures us retail sal...,1,0,0,0,0
2221,kilroy unveils immigration policy ex-chatshow ...,0,0,1,0,0
2222,rem announce new glasgow concert us band rem h...,0,1,0,0,0
2223,how political squabbles snowball it s become c...,0,0,1,0,0


In [ ]:
spasi = re.compile('[/(){}\[\]\|@,;]')
simbol= re.compile('[^0-9a-z #+_]')
referensi = set(stopwords.words('english'))

def bersihkan(text):
    text = text.lower() 
    text = spasi.sub(' ', text) 
    text = simbol.sub('', text) 
    text = text.replace('x', '')
    text = ' '.join(kata for kata in text.split() if kata not in referensi) 
    return text

df_baru["text"] = df_baru["text"].apply(bersihkan)

In [ ]:
plots = df_baru['text'].values
label = df_baru[['business', 'entertainment', 'politics','sport','tech']].values

In [ ]:
from sklearn.model_selection import train_test_split
text_latih, text_test, label_latih, label_test = train_test_split(plots, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
  
tokenizer = Tokenizer(num_words=10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(text_latih) 
tokenizer.fit_on_texts(text_test)
  
sekuens_latih = tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)
  
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=16))
model.add(tf.keras.layers.SpatialDropout1D(0.2))
model.add(tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('val_accuracy')>0.90)&(logs.get('accuracy')>0.90)&(logs.get('val_loss')<0.5) ):
      print("\n akurasi data training dan data validasi telah mencapai 90% dan loss validation dibawah 0,5")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
num_epochs = 30
model.fit(padded_latih, label_latih, 
          epochs=num_epochs, 
          validation_data=(padded_test, label_test), 
          verbose=2, 
          callbacks=[callbacks])

Epoch 1/30
56/56 - 158s - loss: 1.5943 - accuracy: 0.2736 - val_loss: 1.4976 - val_accuracy: 0.3303
Epoch 2/30
56/56 - 160s - loss: 1.4420 - accuracy: 0.3989 - val_loss: 2.8165 - val_accuracy: 0.2517
Epoch 3/30
56/56 - 160s - loss: 1.2713 - accuracy: 0.4433 - val_loss: 1.0583 - val_accuracy: 0.4719
Epoch 4/30
56/56 - 160s - loss: 1.0275 - accuracy: 0.4837 - val_loss: 0.9651 - val_accuracy: 0.5753
Epoch 5/30
56/56 - 160s - loss: 0.9534 - accuracy: 0.5393 - val_loss: 1.0062 - val_accuracy: 0.5326
Epoch 6/30
56/56 - 160s - loss: 0.8807 - accuracy: 0.6152 - val_loss: 0.8755 - val_accuracy: 0.6449
Epoch 7/30
56/56 - 160s - loss: 0.6424 - accuracy: 0.7360 - val_loss: 0.5879 - val_accuracy: 0.7888
Epoch 8/30
56/56 - 159s - loss: 0.4397 - accuracy: 0.8562 - val_loss: 0.6638 - val_accuracy: 0.7371
Epoch 9/30
56/56 - 160s - loss: 0.2526 - accuracy: 0.9348 - val_loss: 0.3926 - val_accuracy: 0.8674
Epoch 10/30

 akurasi data training dan data validasi telah mencapai 90% dan loss validation dibawah